In [ ]:
!pip install tokenizers

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
import time
import torch
import torch.nn as nn

# Transformer로 챗봇을 만들어봅시다!
- Transformer에 문장을 입력하면, 그에 대한 대답을 하는 챗봇을 만들 것입니다.
- 중간중간 빈칸을 채워넣어가며 자신만의 챗봇 코드를 완성해보세요.

우선 챗봇 학습을 위한 데이터를 다운로드 받습니다.

In [ ]:
# data download
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')

In [ ]:
print('데이터 샘플의 개수 :', len(train_data))

In [ ]:
train_data.head(10)

## 한국어 데이터를 전처리하는 방법에 대해 알아봅시다.
- 우리는 네이버 영화리뷰 데이터에 tokenizer를 학습시켜 단어를 구분하도록 할 예정입니다. 
(챗봇 데이터에 비해 훨씬 많은 데이터를 포함하고 있습니다.)

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")
naver_df = pd.read_table('ratings.txt')
naver_df = naver_df.dropna(how='any')
with open('naver_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df['document']))

In [ ]:
from tokenizers import BertWordPieceTokenizer
tokenizer = BertWordPieceTokenizer(lowercase=False) # lowercase를 구분할지 여부를 선택합니다 (True: 대문자 무시(모두 소문자로 인식), False:대소문자 구분)


In [ ]:
data_file = 'naver_review.txt'
vocab_size = 30000
limit_alphabet = 6000
min_frequency = 5

tokenizer.train(files=data_file,
                vocab_size=vocab_size,
                limit_alphabet=limit_alphabet,
                min_frequency=min_frequency,
                special_tokens=['[PAD]', '[START]', '[END]', '[UNK]'])

In [ ]:
# vocab 저장
tokenizer.save_model('./')
vocab_df = pd.read_fwf('vocab.txt', header=None)
vocab_df

In [ ]:
# 챗봇 모델은 "정수 인코딩" 결과를 사용하여 모델을 작동시키고, 최종 예측 결과도 정수로 인코딩 된 결과를 내뱉습니다.
# 인코딩된 결과물을 decode 를 통해 해석 가능한 문장으로 바꿔줄 수 있습니다.
encoded = tokenizer.encode('챗봇이 잘 완성될까요?')
print('토큰화 결과 :',encoded.tokens)
print('정수 인코딩 :',encoded.ids)
print('디코딩 :',tokenizer.decode(encoded.ids))

# Transformer 모델을 만들어봅시다.

In [ ]:
from torch import Tensor
import torch
import torch.nn as nn
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        # ============== TODO ======================
        # torch.nn 에는 Transformer 모델이 이미 구현되어 있습니다. 
        # 이것을 이용해 챗봇 모델을 완성해봅시다.
        # ==========================================
        self.transformer = ????(d_model=emb_size,
                                nhead=nhead,
                                num_encoder_layers=num_encoder_layers,
                                num_decoder_layers=num_decoder_layers,
                                dim_feedforward=dim_feedforward,
                                dropout=dropout)
        # Transformer의 output값을 vocabulary의 index를 나타내는 벡터로 바꾸어주는 선형 변환이 필요합니다.
        self.generator = ????(emb_size, tgt_vocab_size) 
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        # ============== TODO ======================
        # 위에서 구현한 트랜스포머 모델에 입력된 데이터를 통과시켜봅시다.
        # ==========================================
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.????(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.????(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

- Transformer의 학습 중에는, 미래의 데이터를 볼 수 없도록 하는 마스크가 필요합니다.

In [ ]:
PAD_IDX = 0
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [ ]:
SRC_VOCAB_SIZE = vocab_size # 30000
TGT_VOCAB_SIZE = vocab_size # 30000
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

# 모델을 initialize 해줍니다.
model = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)
model = model.to(DEVICE) # model 을 GPU로 보내줍니다.

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=????) # ignore_index 가 하는 역할은 무엇일까요?

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [ ]:
from torch.utils.data import Dataset
class ChatbotDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x, y, _ = self.data[idx]
        # ============== TODO ==============
        # 1. 데이터를 토크나이즈 해보세요. (x,y 모두)
        # 2. start token 과 end token을 추가해보세요. (x,y 모두)
        # 3. 토큰들을 torch tensor (long 타입)로 변환하고 return하세요.
        # ==================================

        x_tokens = ????
        y_tokens = ????

        # hint: x_tokens 와 y_tokens 는 list type입니다.
        x_tokens.????
        x_tokens.????
        y_tokens.????
        y_tokens.????

        return ????, ????

In [ ]:
dataset = ChatbotDataset(train_data.values, tokenizer)

In [ ]:
dataset[0]

In [ ]:
# collate_fn 을 정의해줍니다.
# collate_fn 은 dataset으로부터 여러 item을 받아와 하나의 배치로 합칠 때, 어떻게 합칠지를 정의하는 부분입니다.

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
def collate_fn(batch):
    # batch: [(x1, y1), (x2, y2), ... ]
    # ============== TODO ==============
    # 우리의 목표는 X = [x1,x2,x3,...] , Y = [y1, y2, y3, ...] 형태의 tensor로 만드는 것입니다.
    # 이를 위해 2가지 해야할 일이 있습니다.
    # 1. batch 안에 있는 x 와 y를 각각의 list 에 모아주기.
    # 2. padding을 통해 동일한 길이로 만들어주고, 하나의 tensor로 통합하기.
    # 아래 빈칸을 채워 위 두가지를 진행해보세요.
    # ==================================
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(src_sample)
        tgt_batch.append(tgt_sample)
    
    # padding_value는 pad 위치에 어떤 값을 넣을지를 정하는 값입니다.
    src_batch = ????(src_batch, padding_value=????)
    tgt_batch = ????(tgt_batch, padding_value=????) 
    return src_batch, tgt_batch

# dataloader 를 정의해줍니다.
batch_size = 32
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, 
                         num_workers=2, collate_fn=collate_fn,
                          pin_memory=True, drop_last=True)

In [ ]:
x,y = next(iter(data_loader))

In [ ]:
x.shape, y.shape

In [ ]:
x[:,1]

In [ ]:
from torch.utils.data import DataLoader

def train_epoch(model, optimizer, loss_fn, train_dataloader):
    model.train()
    losses = 0

    for src, tgt in train_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1,:] # 왜 target은 마지막 하나를 빼고 입력할까요?

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :] # 왜 첫번째 단어는 빼고 로스를 계산할까요?
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()
        optimizer.step()
        losses += loss.item()

    return losses / len(train_dataloader)

In [ ]:
EPOCHS = 10
for i in range(EPOCHS):
    epoch_loss = train_epoch(model, optimizer, loss_fn, data_loader)
    print('EPOCH {} LOSS {:.6f}'.format(i+1, epoch_loss))

In [ ]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == 2:
            break
    return ys

def predict(model, tokenizer, src_sentence):
    model.eval()
    src = tokenizer.encode(src_sentence).ids
    src.insert(0,1)
    src.append(2)
    src = torch.tensor(src).long().unsqueeze(1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=1).flatten()
    return tokenizer.decode(tgt_tokens.cpu().tolist())

In [ ]:
predict(model,tokenizer,'반가워')

In [ ]:
predict(model,tokenizer,'여행 가고 싶다.')

# BERT 실습
- BERT는 Transformer의 encoder를 사용합니다.
- Transformer 구현에 대해 이미 알아보았으니, BERT를 직접 구현하지 않고 huggingface 라이브러리를 통해 간단하게 구현하는 방법에 대해 배우도록 하겠습니다.
- BERT를 이용해서 naver 영화리뷰 데이터의 점수를 분류하는 task를 진행해보겠습니다.
- https://huggingface.co/

In [ ]:
!pip install transformers

#### 영화리뷰 데이터에 대한 fine-tuning에 앞서, 간단하게 huggingface 라이브러리의 사용방법에 대해 익혀보도록 합시다.

- https://huggingface.co/transformers/v3.0.2/index.html

In [ ]:
from transformers import BertTokenizer, BertForTokenClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForTokenClassification.from_pretrained('bert-base-uncased')

inputs = tokenizer.encode("Hello, my dog is cute", return_tensors="pt")
outputs = model(inputs)

In [ ]:
outputs

In [ ]:
inputs.shape

- 사전 학습된 BERT의 mask 토큰에 대한 예측 결과를 확인해봅시다.

In [ ]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased')


In [ ]:
unmasker("The man worked as a [MASK].")

In [ ]:
unmasker("The woman worked as a [MASK].")

In [ ]:
unmasker("I have a [MASK].")

- BERT tokenizer와 model을 이용해 네이버 영화리뷰 데이터에 fine-tuning을 진행해봅시다.

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")
naver_df = pd.read_table('ratings.txt')
naver_df = naver_df.dropna(how='any')
with open('naver_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df['document']))

In [ ]:
naver_df.head()

In [ ]:
naver_df.tail()

In [ ]:
naver_df['label'].unique()

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased') # multilingual-BERT를 사용해보겠습니다.
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')

inputs = tokenizer.encode("Hello, my dog is cute", return_tensors="pt")
outputs = model(inputs)

In [ ]:
outputs

In [ ]:
# label을 입력해주면 classification에 대한 loss도 자동으로 계산할 수 있습니다.
model(inputs, labels=torch.tensor([1]))

In [ ]:
train_data_idx = np.random.choice(range(len(naver_df)), size=len(naver_df)//5*4, replace=False)

In [ ]:
train_data_idx

In [ ]:
train_data = naver_df.iloc[train_data_idx][['document','label']].values
test_data = naver_df.iloc[~naver_df.index.isin(train_data_idx)][['document','label']].values

In [ ]:
from torch.utils.data import Dataset
class ReviewDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x, y = self.data[idx]
        # BERT tokenizer는 batch 단위로 한번에 token화 할 수 있습니다.
        # dataloder에서 얻어온 후 한번에 tokenize 해보도록 하겠습니다.
        # text는 list로, label은 long tensor로 리턴해줍니다.
        return x, torch.tensor(y).long()

In [ ]:
train_dataset = ReviewDataset(train_data)
test_dataset = ReviewDataset(test_data)

In [ ]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, 
                         num_workers=2, collate_fn=None,
                          pin_memory=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, 
                         num_workers=2, collate_fn=None,
                          pin_memory=True, drop_last=False)

In [ ]:
x, y = next(iter(train_loader))

In [ ]:
x

In [ ]:
y

In [ ]:
encoded_x = tokenizer.batch_encode_plus(x, padding=True, return_tensors='pt')

In [ ]:
encoded_x.keys()

In [ ]:
encoded_x['input_ids']

In [ ]:
encoded_x['token_type_ids']

In [ ]:
encoded_x['attention_mask']

In [ ]:
def train_epoch(model, dataloader, tokenizer, optimizer):
    model.train()
    train_loss = 0
    for i, (x,y) in enumerate(dataloader):
        # =========== TODO ============
        # 위에서 배운 내용을 바탕으로 빈칸을 채워보세요!
        # =============================
        x = tokenizer.????(x, padding=True, return_tensors='pt')['input_ids'].to(DEVICE)
        y = y.to(DEVICE)
        loss = model(x, labels=y)[????]
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        if i % 50 == 0:
            print('Iter [{}/{}] Loss {:.6f}'.format(i+1, len(dataloader), train_loss / (i+1)))
    
    return train_loss / len(dataloader)

def test_epoch(model, dataloader, tokenizer):
    model.eval()
    preds = []
    labels = []
    with torch.no_grad():
      for x,y in dataloader:
          # =========== TODO ============
          # 위에서 배운 내용을 바탕으로 빈칸을 채워보세요!
          # =============================
          x = tokenizer.????(x, padding=True, return_tensors='pt')['input_ids'].to(DEVICE)
          out = model(x)[????]
          pred = out.argmax(-1)
          preds.append(pred.cpu())
          labels.append(y)
    preds = torch.cat(preds)
    labels = torch.cat(labels)
    acc = (preds == labels).float().mean()
    print('ACC : {:.3f}'.format(acc))
    return preds, labels

def predict(model, tokenizer, sentence):
    model.eval()
    x = tokenizer.encode(sentence, return_tensors='pt').to(DEVICE)
    out = model(x)['logits']
    pred = out.argmax(-1)
    return pred.cpu()

In [ ]:

model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

In [ ]:
EPOCHS=1

for i in range(EPOCHS):
    train_epoch(model, train_loader, tokenizer, optimizer)
    test_epoch(model, test_loader, tokenizer)

In [ ]:
predict(model, tokenizer, '이 영화는 최고야')

In [ ]:
predict(model, tokenizer, '이 영화는 별로야')